In [ ]:
%pip install -qU datasets

In [4]:
%pip install -qU langchain-groq
%pip install -qU json5
%pip install langchain
%pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 344.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.6 MB/s eta 0:00:00


In [9]:
from google.colab import userdata
groq_key = userdata.get('GROQ_API_KEY')

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    groq_api_key=groq_key,
    # other params...
)

In [10]:
import os
import json5 as json
import pandas as pd
from time import sleep
from datasets import load_dataset
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

In [6]:
def get_extracted_json(str_data):
# Define the template for extracting information
    template = """
    You are an assistant that extracts information from invoices. Given the following text input, extract the Customer Name, Invoice No., Invoice Date, and Total Amount. Return the extracted information in a JSON format as shown below:

    Input:
    {invoice_text}

    Output format:
    {{
      "Customer Name": {{
        "value": "<Customer Name>"
      }},
      "Invoice No.": {{
        "value": "<Invoice No.>"
      }},
      "Invoice Date": {{
        "value": "<Invoice Date>"
      }},
      "Total Amount": {{
        "value": "<Total Amount>"
      }}
    }}

    Example Input:
    "Invoice no: 40378170 Date of issue: Seller: Patel, Thompson and Montgomery 356 Kyle Vista New James, MA 46228 Tax Id: 958-74-3511 IBAN: GB77WRBQ31965128414006 ITEMS No. Description Qty 1. Leed's Wine Companion Bottle 1,00 Corkscrew Opener Gift Box Set with Foil Cutter SUMMARY VAT [%] 10% Total 10/15/2012 Client: Jackson, Odonnell and Jackson 267 John Track Suite 841 Jenniferville, PA 98601 Tax Id: 998-87-7723 UM Net price Net worth VAT [%] each 7,50 7,50 10% Net worth VAT 7,50 0,75 $ 7,50 $ 0,75 Gross worth 8,25 Gross worth 8,25 $ 8,25"

    Example Output:
    {{
      "Customer Name": {{
        "value": "Jackson, Odonnell and Jackson"
      }},
      "Invoice No.": {{
        "value": "40378170"
      }},
      "Invoice Date": {{
        "value": "10/15/2012"
      }},
      "Total Amount": {{
        "value": "$ 8,25"
      }}
    }}

    Now, provide the output for the given input.
    """
    prompt = PromptTemplate(
        input_variables=["invoice_text"],
        template=template,
    )

    chain = LLMChain(prompt=prompt, llm=llm)

    invoice_text = str_data
    result = chain.run(invoice_text=invoice_text)

    return result


In [7]:
def extract_json(text):
  """Extracts JSON data from a text string.

  Args:
      text (str): The text string containing the JSON data.

  Returns:
      dict: The parsed JSON data as a dictionary.

  Raises:
      json.JSONDecodeError: If the JSON data is invalid.
    """
  start = text.find('{')
  end = text.rfind('}') + 1
  json_string = text[start:end]
  return json.loads(json_string)

In [8]:
def update_json(json_data, coordinates):
  """Updates a JSON object with coordinate information.

  Args:
    json_data: The JSON object to be updated.
    coordinates: A dictionary mapping values to their corresponding coordinates.

  Returns:
    The updated JSON object.
  """

  updated_json = {}
  for key, value in json_data.items():
    updated_value = value.copy()  # Create a copy to avoid modifying original
    if key != 'Invoice No.':
      updated_value['coordinate'] = coordinates.get(value['value'], "NA")
    else:
      invoice = f"Invoice no: {value['value']}"
      updated_value['coordinate'] = coordinates.get(invoice, "NA")
    updated_json[key] = updated_value
  return updated_json

In [9]:
def create_and_save_json_df(input1, input2, filename):
    """Reads a CSV file, processes each row starting from the specified start_row, and extracts JSON data from the 'context' column.

    Args:
      filename: The name of the CSV file to read.
      start_row: The starting row index from which to begin processing.

    Returns:
      None
    """

    if os.path.exists(filename):
      df = pd.read_csv(filename)
    else:
      df = pd.DataFrame(columns=['index', 'json_data'])
    df.loc[len(df)] = [input1, input2]
    df.to_csv(filename, index=False)

In [12]:
def read_csv_with_dict(filename, start_row):

  """Reads a CSV file, processes each row starting from the specified start_row, and extracts JSON data from the 'context' column.

  Args:
    filename: The name of the CSV file to read.
    start_row: The starting row index from which to begin processing.

  Returns:
    None
  """
  df = pd.read_csv(filename)
  for index, row in df.iterrows():
    if index >= start_row:
      index_value = row['index']
      context = row['context']
      context_with_coordinates = eval(row['context_with_coordinates'])
      json_result = get_extracted_json(context)
      json_data = extract_json(json_result)
      updated_json = update_json(json_data, context_with_coordinates)
      create_and_save_json_df(index_value, updated_json, "groq_json_results.csv")
      sleep(2)
read_csv_with_dict("ocr_results.csv", 0)


In [ ]:
ds = load_dataset('katanaml-org/invoices-donut-data-v1')

In [30]:
data_part = "train"
start_row = 0
end_row = 100
df_data = ds[data_part].to_pandas()
df_data.drop('image', axis=1, inplace=True)
df_data = df_data.iloc[start_row:end_row]
df_data.reset_index(inplace=True)
merged_df = pd.merge(df_data, pd.read_csv('groq_json_results.csv'), on='index')
merged_df['ground_truth'] = merged_df['ground_truth'].apply(json.loads)
merged_df['json_data'] = merged_df['json_data'].apply(json.loads)
merged_df.to_csv('final_results_groq.csv', index=False)

In [36]:
merged_df.head()

,index,ground_truth,json_data
0,0,{'gt_parse': {'header': {'invoice_no': '403781...,"{'Customer Name': {'value': 'Jackson, Odonnell..."
1,1,{'gt_parse': {'header': {'invoice_no': '613562...,{'Customer Name': {'value': 'Rodriguez-Stevens...
2,2,{'gt_parse': {'header': {'invoice_no': '495650...,"{'Customer Name': {'value': 'Garcia Inc', 'coo..."
3,3,{'gt_parse': {'header': {'invoice_no': '956116...,"{'Customer Name': {'value': 'Johnson Group', '..."
4,4,{'gt_parse': {'header': {'invoice_no': '260200...,"{'Customer Name': {'value': 'Ochoa, Crane and ..."


In [32]:
comparison_list = []

for index, row in merged_df.iterrows():
    json_data = row['json_data']
    if json_data:
        invoice_number_1 = json_data.get('Invoice No.', {}).get('value', '')
        invoice_date_1 = json_data.get('Invoice Date', {}).get('value', '')
        client_name_1 = json_data.get('Customer Name', {}).get('value', '')
        gross_worth_1 = json_data.get('Total Amount', {}).get('value', '')
    else:
        invoice_number_1 = ''
        invoice_date_1 = ''
        client_name_1 = ''
        gross_worth_1 = ''

    ground_truth = row['ground_truth']
    if ground_truth:
        invoice_number_2 = ground_truth.get('gt_parse', {}).get('header', {}).get('invoice_no', '')
        invoice_date_2 = ground_truth.get('gt_parse', {}).get('header', {}).get('invoice_date', '')
        client_name_2 = ground_truth.get('gt_parse', {}).get('header', {}).get('client', '')
        gross_worth_2 = ground_truth.get('gt_parse', {}).get('summary', {}).get('total_gross_worth', '')
    else:
        invoice_number_2 = ''
        invoice_date_2 = ''
        client_name_2 = ''
        gross_worth_2 = ''

    comparison_results = {
        'index': index,
        'Customer Name Matches': client_name_1 in client_name_2,
        'Invoice Number Matches': invoice_number_1 == invoice_number_2,
        'Invoice Date Matches': invoice_date_1 == invoice_date_2,
        'Total Amount Matches': gross_worth_1.strip().replace('$ ', '').replace('$','').replace(' ', '').replace('.', ',') == gross_worth_2.strip().replace('$ ', '').replace('$','').replace(' ', '').replace('.', ',')
    }

    # Add comparison results to the list
    comparison_list.append(comparison_results)

    # Output the details
    print(f"Invoice Number ocr, {index}:", invoice_number_1)
    print(f"Invoice Date ocr, {index}:", invoice_date_1)
    print(f"Client Name ocr, {index}:", client_name_1)
    print(f"Gross Worth ocr, {index}:", gross_worth_1)
    print(f"Invoice Number, {index}:", invoice_number_2)
    print(f"Invoice Date, {index}:", invoice_date_2)
    print(f"Client Name, {index}:", client_name_2)
    print(f"Gross Worth, {index}:", gross_worth_2)
    print("----" * 30)

# Convert comparison results to DataFrame
comparison_df = pd.DataFrame(comparison_list)

# Calculate percentage matches for each field
total_comparisons = len(comparison_df)
percentage_matches = {
    'Customer Name Matches (%)': (comparison_df['Customer Name Matches'].sum() / total_comparisons) * 100,
    'Invoice Number Matches (%)': (comparison_df['Invoice Number Matches'].sum() / total_comparisons) * 100,
    'Invoice Date Matches (%)': (comparison_df['Invoice Date Matches'].sum() / total_comparisons) * 100,
    'Total Amount Matches (%)': (comparison_df['Total Amount Matches'].sum() / total_comparisons) * 100
}

print("\nComparison Results Summary:")
for key, value in percentage_matches.items():
    print(f"{key}: {value:.2f}%")

Invoice Number ocr, 0: 40378170
Invoice Date ocr, 0: 10/15/2012
Client Name ocr, 0: Jackson, Odonnell and Jackson
Gross Worth ocr, 0: $ 8,25
Invoice Number, 0: 40378170
Invoice Date, 0: 10/15/2012
Client Name, 0: Jackson, Odonnell and Jackson 267 John Track Suite 841 Jenniferville, PA 98601
Gross Worth, 0: $8,25
------------------------------------------------------------------------------------------------------------------------
Invoice Number ocr, 1: 61356291
Invoice Date ocr, 1: 09/06/2012
Client Name ocr, 1: Rodriguez-Stevens
Gross Worth ocr, 1: $ 212,09
Invoice Number, 1: 61356291
Invoice Date, 1: 09/06/2012
Client Name, 1: Rodriguez-Stevens 2280 Angela Plain Hortonshire, MS 93248
Gross Worth, 1: $ 212,09
------------------------------------------------------------------------------------------------------------------------
Invoice Number ocr, 2: 49565075
Invoice Date ocr, 2: 10/28/2019
Client Name ocr, 2: Garcia Inc
Gross Worth ocr, 2: $ 96.73
Invoice Number, 2: 49565075
Invoice

In [39]:
from sklearn.metrics import confusion_matrix
from typing import Dict
import pandas as pd

def flatten_json(json_obj: Dict) -> Dict:
    """
    Flattens a nested JSON object into a single-level dictionary.

    Args:
        json_obj (Dict): The nested JSON object to flatten.

    Returns:
        Dict: The flattened JSON object as a single-level dictionary.
    """
    flat_dict = {}

    def flatten(data, parent_key=''):
        if isinstance(data, dict):
            for k, v in data.items():
                new_key = f"{parent_key}{k}"
                if isinstance(v, dict) or isinstance(v, list):
                    flatten(v, new_key + '_')
                else:
                    flat_dict[new_key] = v
        elif isinstance(data, list):
            for i, item in enumerate(data):
                flatten(item, parent_key + str(i) + '_')

    flatten(json_obj)
    return flat_dict

def evaluate_json(ground_truth: Dict, predicted: Dict) -> Dict:
    """Evaluates the accuracy of a predicted JSON object against a ground truth JSON object.

    Args:
        ground_truth (Dict): The ground truth JSON object.
        predicted (Dict): The predicted JSON object.

    Returns:
        Dict: A dictionary containing the following metrics:
            - True Positive: Number of correctly predicted values.
            - False Positive: Number of incorrectly predicted values.
            - True Negative: Number of correctly predicted absences.
            - False Negative: Number of incorrectly predicted absences.
            - value_accuracy: Overall accuracy based on the number of matching values.
    """
    metrics = {'True Positive': 0, 'False Positive': 0, 'True Negative': 0, 'False Negative': 0}

    gt_flat = flatten_json(ground_truth)
    pred_flat = flatten_json(predicted)

    # Define matching criteria
    matches = {
        'Customer Name Matches': pred_flat.get('Customer Name_value', 'NA') in gt_flat.get('gt_parse_header_client', 'NA'),
        'Invoice Number Matches': pred_flat.get('Invoice No._value', 'NA') == gt_flat.get('gt_parse_header_invoice_no', 'NA'),
        'Invoice Date Matches': pred_flat.get('Invoice Date_value', 'NA') == gt_flat.get('gt_parse_header_invoice_date', 'NA'),
        'Total Amount Matches': (
            pred_flat.get('Total Amount_value', 'NA').strip().replace('$ ', '').replace('$', '').replace(' ', '').replace('.', ',') ==
            gt_flat.get('gt_parse_summary_total_gross_worth', 'NA').strip().replace('$ ', '').replace('$', '').replace(' ', '').replace('.', ',')
        )
    }


    groundt_truth = {'Customer Name Matches':'gt_parse_header_client', 'Invoice Number Matches':'gt_parse_header_invoice_no','Invoice Date Matches': 'gt_parse_header_invoice_date', 'Total Amount Matches':'gt_parse_summary_total_gross_worth'}
    predict_val = {'Customer Name Matches':'Customer Name_value', 'Invoice Number Matches':'Invoice No._value','Invoice Date Matches': 'Invoice Date_value', 'Total Amount Matches':'Total Amount_value'}
    # Calculate confusion matrix components based on matching criteria
    for match_type, is_match in matches.items():
        gt_value_present = gt_flat.get(groundt_truth.get(match_type), 'NA') != 'NA'
        pred_value_present = pred_flat.get(predict_val.get(match_type), 'NA') != 'NA'

        if gt_value_present and pred_value_present and is_match:
            # True Positive: Both values are present and match
            metrics['True Positive'] += 1
        elif not pred_value_present and gt_value_present or not is_match:
            # False Positive: Predicted value not present, but ground truth value is present
            metrics['False Positive'] += 1
        elif pred_value_present and not gt_value_present:
            # False Negative: Predicted value present, but ground truth value is not present
            metrics['False Negative'] += 1
        elif not gt_value_present and not pred_value_present:
            # True Negative: Both values are not present
            metrics['True Negative'] += 1

    # Compute accuracy
    total_matches = len(matches)
    value_accuracy = (metrics['True Positive'] / total_matches) * 100 if total_matches > 0 else 0
    metrics['value_accuracy'] = value_accuracy

    # print(metrics)
    return metrics

# Initialize accumulators
total_confusion = {'True Positive': 0, 'False Positive': 0, 'True Negative': 0, 'False Negative': 0}
total_values = 0
total_correct = 0

# Iterate over the DataFrame rows
for index, row in merged_df.iterrows():
    ground_truth = row['ground_truth']
    predicted = row['json_data']
    metrics = evaluate_json(ground_truth, predicted)

    # Accumulate results
    total_confusion['True Positive'] += metrics['True Positive']
    total_confusion['False Positive'] += metrics['False Positive']
    total_confusion['True Negative'] += metrics['True Negative']
    total_confusion['False Negative'] += metrics['False Negative']
    total_values += len(flatten_json(row['json_data']))
    total_correct += metrics['value_accuracy'] * len(flatten_json(row['json_data'])) / 100

# Compute aggregate metrics
aggregated_metrics = {
    'True Positive': total_confusion['True Positive'],
    'False Positive': total_confusion['False Positive'],
    'True Negative': total_confusion['True Negative'],
    'False Negative': total_confusion['False Negative'],
    'Total Values': total_values,
    'Total Correct': total_correct,
    'Overall Accuracy (%)': (total_correct / total_values) * 100
}

# Display aggregated results
aggregated_results_df = pd.DataFrame([aggregated_metrics])
print(aggregated_results_df)


   True Positive  False Positive  True Negative  False Negative  Total Values  \
0            367              33              0               0           453   

   Total Correct  Overall Accuracy (%)  
0         410.75             90.673289  
